# Parsers

In [ ]:
import numpy as np
import pandas as pd

from src.parsers import parse_reporting_info, parse_closed_banks

In [ ]:
REPORTING_DATA_FOLDER = 'parsed_data/bank_reporting/'
CLOSED_BANKS_DATA_FOLDER = 'parsed_data/closed_banks/'

# Список из ID страничек и соответствующие им названия переменных. 
ID_list = ['10','30','25','20','40','50','60','500','1000','1100','1200','1300','1400','1500','1550',\
           '1600','1700','1800','200','260','300','360','120','70','110','130','140','160','400','401']

ID_names_list = ['na','ni','c_123','c_134','cp','LLPGL','dep_f','dep_b','ROA','ROE','LLP','IR','LCP','fr_na',\
                 'fr','H1','H2','H3','c_f','LLP_c_f','c_b','LLP_c_b','ibl','sec_tot','liquid','sec','bond','oth_cap',\
                 'retail','retail_fr']

### 1. Bank reporting info

In [ ]:
# Сгенерируем лист из дат, которые необходимо пробежать.
# февраль 17 - март 08  (2017-02-01 до  2008-03-01)
years = [str(item) for item in range(2010,2018)]
months = [str(item) for item in range(1,13)]
months2 = [ ]
for item in months:
    if len(item)==1:
        item = '0'+item
    months2.append(item)
dats = [ ]
for item in years:
    for jtem in months2:
        dats.append(item + '-' + jtem + '-01')

In [ ]:
parse_reporting_info(ID_list, dats, path=REPORTING_DATA_FOLDER)
parse_closed_banks(path=CLOSED_BANKS_DATA_FOLDER)

In [ ]:
# Для начала неплохо было бы выявить список из уникальных банков! 

# Функция, которая обхелиняет два лоиста по уникалтным названиям 
# Она делает пересечение двух множеств 
def union(a, b):
    return list(set(a) | set(b))

def Uniq_banks(frames):
    uniq_banks = list(frames[0]['Лицензия_банк'])
    for item in ID_list:
        uniq_banks = union(uniq_banks,item)
    return(uniq_banks)

# Сохраним ко всему прочему информацию о том какие банки уникальны.
uniq_banks = Uniq_banks(frames)
un_banks = pd.DataFrame()
un_banks['Уникальные банки'] = uniq_banks

# Делаем список из датафреймов, которые мы подгружаем из папки. Соединим все показатели в один датафрейм
frames = [pd.read_csv(REPORTING_DATA_FOLDER + strtem +'.csv', header=0, sep=',', encoding="cp1251") 
          for strtem in ID_list]
df_all = frames[0].set_index(['лицензия', 'дата'])
    
i = 1
while i <= len(frames):
    df_tmp = pd.DataFrame(frames[i])
    df_all = df_all.join(df_tmp.set_index(['лицензия', 'дата']), how = 'outer')
    df_all.drop_duplicates(keep = 'first', inplace =True)
    i = i + 1

In [ ]:
# функция для выделения номера лицензии из строки
def get_license(string):
    license = string.split('№ ')[1].split(',')[0]
    return license

bank_frame['лицензия'] = bank_frame['Лицензия_банк'].apply(get_license)

In [ ]:
df_all.to_csv(REPORTING_DATA_FOLDER + 'reporting_all.csv', sep=',',header=True,index=True)
un_banks.to_csv(REPORTING_DATA_FOLDER + 'unique_banks.csv',sep='\t',header=True,index=False)

### 2. Closed banks info

In [ ]:
parse_closed_banks(path=CLOSED_BANKS_DATA_FOLDER, mainpage='http://www.banki.ru')

In [ ]:
closed_banks = pd.read_csv('closed_banks.csv', header=0, sep=',', encoding="cp1251")
closed_banks['ликвдата'] = pd.to_datetime(closed_banks['ликвдата'])

# Причины по ликвидированным банкам: 
likvid_data = closed_banks[closed_banks['статус']=='ликв.']
otozv_data = closed_banks[closed_banks['статус']=='отозв.']

likvid_data.to_csv(CLOSED_BANKS_DATA_FOLDER + 'likvid_data.csv',sep='\t',header=True,index=False)
otozv_data.to_csv(CLOSED_BANKS_DATA_FOLDER + 'otozv_data.csv',sep='\t',header=True,index=False)

### 3. Other data

Other data was already in the form of .csv at Central Bank website